## Import useful stuff

In [1]:
%pylab
%load_ext autoreload
%autoreload 2

from datetime import datetime, timedelta

import os
import sys

import numpy as np
import pandas as pd
from scipy import sparse
import sklearn as sl
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn_pandas import DataFrameMapper
from sklearn.base import TransformerMixin
from gensim.models import Word2Vec

from spellchecker import init_nwords, correct

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


# Load train data
Using pandas' read_csv with all the defaults

In [2]:
if os.name == 'nt':
    TRAIN_PATH = r'D:\train.csv'
    PTRAIN_PATH = r'D:\train_preprocessed.csv'
    TEST_PATH = r'D:\test.csv'
    FREEBASE_PATH = r'D:\freebase-vectors-skipgram1000.bin.gz'
    GOOGNEWS_PATH = r'D:\GoogleNews-vectors-negative300.bin.gz'
    VOCAB_PATH = r'D:\big.txt'
# df = pd.read_csv(TRAIN_PATH, low_memory=False)
df = pd.read_csv(PTRAIN_PATH)
#df

### Define columns

In [20]:
bool_cols = ['VAR_0008', 'VAR_0009', 'VAR_0010',
             'VAR_0011', 'VAR_0012', 'VAR_0043',
             'VAR_0196', 'VAR_0226', 'VAR_0229',
             'VAR_0230', 'VAR_0232', 'VAR_0236',
             'VAR_0239']
cleaned_bool_cols = [c for c in bool_cols if c in df.columns]

date_cols = ['VAR_0073', 'VAR_0075', 'VAR_0156',
             'VAR_0157', 'VAR_0158', 'VAR_0159',
             'VAR_0166', 'VAR_0167', 'VAR_0168',
             'VAR_0169', 'VAR_0176', 'VAR_0177',
             'VAR_0178', 'VAR_0179', 'VAR_0204',
             'VAR_0217']
cleaned_date_cols = [c for c in date_cols if c in df.columns]

str_cols = [u'VAR_0001', u'VAR_0005', u'VAR_0044',
            u'VAR_0200', u'VAR_0202', u'VAR_0214',
            u'VAR_0216', u'VAR_0222', u'VAR_0237',
            u'VAR_0274', u'VAR_0283', u'VAR_0305',
            u'VAR_0325', u'VAR_0342', u'VAR_0352',
            u'VAR_0353', u'VAR_0354', u'VAR_0404',
            u'VAR_0466', u'VAR_0467', u'VAR_0493',
            u'VAR_1934']
cleaned_str_cols = [c for c in str_cols if c in df.columns]

other_cols = set(bool_cols + date_cols + str_cols)
# Column VAR_0044 contains only '[]'
# Columns VAR_0218 and VAR_0240 are missing in the training data
float_cols = ['VAR_%04d'%i for i in range(1, 1935) if ('VAR_%04d'%i) not in other_cols and i not in (44, 218, 240)]
cleaned_float_cols = [c for c in float_cols if c in df.columns]


##Cast bool columns as bool
Also impute values by replacing missing data with most frequent value per column.

In [4]:
def clean_bools(bool_val):
    if str(bool_val) not in ('True', 'False'):
        return np.nan
    else:
        return bool_val

df[bool_cols] = df[bool_cols].applymap(clean_bools)

most_common_bools = df[bool_cols].describe().loc['top', :]
df[bool_cols] = df[bool_cols].fillna(value=most_common_bools)
df[bool_cols] = df[bool_cols].astype(np.bool)
df[bool_cols]

KeyError: "['VAR_0008' 'VAR_0009' 'VAR_0010' 'VAR_0011' 'VAR_0012' 'VAR_0043'\n 'VAR_0196' 'VAR_0229' 'VAR_0239'] not in index"

## Parse the weird date format of the date column
Also impute values by replacing missing data with most frequent value per column.

In [4]:
def parse_date_str(date_val):
    if isinstance(date_val, datetime):
        return date_val
    try:
        return datetime.strptime(date_val, '%d%b%y:%H:%M:%S')
    except:
        return np.nan

most_common_dates = df[date_cols].describe().loc['top', :]
df[date_cols] = df[date_cols].fillna(value=most_common_dates)
df[date_cols] = df[date_cols].applymap(parse_date_str).astype('datetime64[ns]')
df[date_cols]

,VAR_0073,VAR_0075,VAR_0156,VAR_0157,VAR_0158,VAR_0159,VAR_0166,VAR_0167,VAR_0168,VAR_0169,VAR_0176,VAR_0177,VAR_0178,VAR_0179,VAR_0204,VAR_0217
0,2009-03-13,2011-11-08,2011-12-12,2012-07-27,2012-02-01,2012-06-04,2011-12-19,2011-11-03,2011-11-03,2011-12-19,2011-12-12,2011-11-03,2011-11-03,2012-06-04,2014-01-29 21:16:00,2011-11-08 02:00:00
1,2012-09-04,2011-11-10,2011-12-12,2012-07-27,2012-02-01,2012-06-04,2011-12-19,2011-11-03,2011-11-03,2011-12-19,2011-12-12,2011-11-03,2011-11-03,2012-06-04,2014-02-01 00:11:00,2012-10-02 02:00:00
2,2009-03-13,2011-12-13,2011-12-12,2012-07-27,2012-02-01,2012-06-04,2011-12-19,2011-11-03,2011-11-03,2011-12-19,2011-12-12,2011-11-03,2011-11-03,2012-06-04,2014-01-30 15:11:00,2011-12-13 02:00:00
3,2009-03-13,2010-09-23,2011-12-12,2012-07-27,2012-02-01,2012-06-04,2011-12-19,2011-11-03,2011-11-03,2011-12-19,2011-12-12,2011-11-03,2011-11-03,2012-06-04,2014-02-01 00:07:00,2012-11-01 02:00:00
4,2009-03-13,2011-10-15,2011-12-12,2012-07-27,2012-02-01,2012-06-04,2011-12-19,2011-11-03,2011-11-03,2011-12-19,2011-12-12,2011-11-03,2011-11-03,2012-06-04,2014-01-29 19:31:00,2011-10-15 02:00:00
5,2009-03-13,2012-06-06,2011-12-12,2012-07-27,2012-02-01,2012-06-04,2011-12-19,2011-11-03,2011-11-03,2011-12-19,2011-12-12,2011-11-03,2011-11-03,2012-06-04,2014-01-30 23:52:00,2012-06-06 02:00:00
6,2009-03-13,2010-11-26,2011-12-12,2012-07-27,2012-02-01,2012-06-04,2011-12-19,2011-11-03,2011-11-03,2011-12-19,2011-12-12,2011-11-03,2011-11-03,2012-06-04,2014-01-29 19:18:00,2011-10-11 02:00:00
7,2009-03-13,2012-01-18,2011-12-12,2012-07-27,2012-02-01,2012-06-04,2011-12-19,2011-11-03,2011-11-03,2011-12-19,2011-12-12,2011-11-03,2011-11-03,2012-06-04,2014-01-30 16:46:00,2012-01-18 02:00:00
8,2012-01-26,2012-01-26,2011-12-12,2012-07-27,2012-02-01,2012-06-04,2012-03-12,2011-11-03,2012-03-22,2012-03-12,2012-03-12,2011-11-03,2012-03-22,2012-03-12,2014-01-30 21:39:00,2012-04-23 02:00:00
9,2009-03-13,2012-07-28,2011-12-12,2012-07-27,2012-02-01,2012-06-04,2011-12-19,2011-11-03,2011-11-03,2011-12-19,2011-12-12,2011-11-03,2011-11-03,2012-06-04,2014-01-31 17:40:00,2012-07-28 02:00:00


## Cast string columns as string and make no data uniform (instead of nan, -1, [], etc.)
Also impute values by replacing missing data with most frequent value per column.

In [5]:
def filter_str(str_cell):
    str_cell = str(str_cell).strip().lower()
    if str_cell in ('-1', '[]', 'nan'):
        return ''
    else:
        return str_cell

most_common_str = df[str_cols].describe().loc['top', :]
df[str_cols] = df[str_cols].fillna(value=most_common_str)
df[str_cols] = df[str_cols].astype(np.str).applymap(filter_str)
df[str_cols]

## Impute missing float values

In [8]:
df[float_cols] = df[float_cols].astype(np.float64)
col_means = df[float_cols].describe().loc['mean', :]
df[float_cols] = df[float_cols].fillna(value=col_means)
df[float_cols]

,VAR_0002,VAR_0003,VAR_0004,VAR_0006,VAR_0007,VAR_0013,VAR_0014,VAR_0015,VAR_0016,VAR_0017,...,VAR_1924,VAR_1925,VAR_1926,VAR_1927,VAR_1928,VAR_1929,VAR_1930,VAR_1931,VAR_1932,VAR_1933
0,224,0,4300,0,0,0,0,0,1,0,...,9998,0,98,98,998,999999998,998,998,9998,9998
1,7,53,4448,1,0,1,0,1,2,1,...,9998,0,98,98,998,999999998,998,998,9998,9998
2,116,3,3464,0,0,0,0,0,1,0,...,9998,0,98,98,998,999999998,998,998,9998,9998
3,240,300,3200,0,0,0,0,0,2,0,...,9998,0,98,98,998,999999998,998,998,9998,9998
4,72,261,2000,0,0,0,0,0,1,0,...,9998,0,98,98,998,999999998,998,998,9998,9998
5,4,4,4422,0,0,0,0,0,1,0,...,9998,0,98,98,998,999999998,998,998,9998,9998
6,60,132,40000,1,1,1,1,1,2,0,...,9998,1,0,1,0,999999996,400,400,9996,76
7,13,75,3600,0,0,0,0,0,1,0,...,9998,0,98,98,998,999999998,998,998,9998,9998
8,17,16,2296,1,1,1,1,1,1,1,...,9998,1,0,97,997,999999997,400,997,9996,90
9,24,72,450,0,0,0,0,0,1,0,...,9998,0,98,98,998,999999998,998,998,9998,9998


# Vectorize String and Datetime colums

## String columns

### Correct typos and mispellings

In [4]:
columns_with_words = ['VAR_0200', 'VAR_0404', 'VAR_0467', 'VAR_0493', 'VAR_1934']

In [8]:
init_nwords(VOCAB_PATH)
def correct_sentence(str_val):
    return ' '.join(correct(w) for w in str_val.split())

df[columns_with_words] = df[columns_with_words].applymap(correct_sentence)
df.to_csv(PTRAIN_PATH, index=False)

### Vectorize columns with more than 300 values (VAR_0493, VAR_0404, VAR_0200)
#### Load pre-trained Word2Vec GoogleNews model

In [5]:
model = Word2Vec.load_word2vec_format(GOOGNEWS_PATH, binary=True)

#### Create new Vectorize columns and c

In [8]:
cols_to_vectorize = ['VAR_0493', 'VAR_0404', 'VAR_0200']

In [ ]:
vector_size = model.syn0.shape[1]

def vectorize_str_row(str_row):
    ret_val = np.zeros(len(str_row) * vector_size)
    for i, str_cell in enumerate(str_row):
        split_str = [s for s in str_cell.split() if s in model.vocab]
        if split_str:
            ret_val[i*vector_size:(i+1)*vector_size] = np.sum(model[s] for s in split_str)
    return ret_val

vectorized_str_cols = pd.DataFrame(columns=['%s_%03d'%(c, i) for c in cols_to_vectorize for i in range(vector_size)],
                                  index=df.index)
vectorized_str_cols.loc[:, :] = np.apply_along_axis(vectorize_str_row, 1, np.asarray(df.loc[:, cols_to_vectorize]))
#for i, c in enumerate(cols_to_vectorize):
#    column_mask = slice(i*vector_size, (i+1)*vector_size)
#    print vectorized_str_cols.iloc[:, column_mask].shape
#    print np.apply_along_axis(vector_repr, 1, np.asarray(df[c])).shape
#    break

In [23]:
vectorized_str_cols.describe()

,VAR_0493_000,VAR_0493_001,VAR_0493_002,VAR_0493_003,VAR_0493_004,VAR_0493_005,VAR_0493_006,VAR_0493_007,VAR_0493_008,VAR_0493_009,...,VAR_0200_290,VAR_0200_291,VAR_0200_292,VAR_0200_293,VAR_0200_294,VAR_0200_295,VAR_0200_296,VAR_0200_297,VAR_0200_298,VAR_0200_299
count,145231.000000,145231.000000,145231.000000,145231.000000,145231.000000,145231.000000,145231.000000,145231.000000,145231.000000,145231.000000,...,145231.000000,145231.000000,145231.000000,145231.000000,145231.000000,145231.000000,145231.000000,145231.000000,145231.000000,145231.000000
mean,-0.018819,-0.123239,0.004211,-0.025742,-0.102731,-0.018499,-0.042962,-0.103002,0.095500,-0.137647,...,0.004844,0.019825,-0.021290,0.019829,-0.006282,-0.024798,-0.028572,-0.031101,0.001024,0.018961
std,0.026909,0.029282,0.014324,0.017882,0.023293,0.028111,0.026438,0.016495,0.024635,0.029572,...,0.053627,0.051655,0.056964,0.048495,0.047026,0.053838,0.058783,0.050528,0.058776,0.054429
min,-0.320982,-0.315228,-0.304574,-0.258437,-0.329134,-0.495254,-0.192985,-0.440000,-0.245863,-0.384118,...,-0.307163,-0.237034,-0.393424,-0.229563,-0.266324,-0.308008,-0.333347,-0.262091,-0.442648,-0.305617
25%,-0.018429,-0.127093,0.004831,-0.027790,-0.107139,-0.023384,-0.048180,-0.103645,0.099420,-0.142736,...,-0.011557,0.000000,-0.043291,0.000000,-0.029382,-0.056473,-0.061861,-0.065501,-0.029266,0.000000
50%,-0.018429,-0.127093,0.004831,-0.027790,-0.107139,-0.023384,-0.048180,-0.103645,0.099420,-0.142736,...,0.000000,0.000000,0.000000,0.000000,0.000000,-0.003525,-0.006685,-0.017339,0.000000,0.000000
75%,-0.018429,-0.127093,0.004831,-0.027790,-0.107139,-0.023384,-0.048180,-0.103645,0.099420,-0.142736,...,0.029552,0.060760,0.000000,0.048598,0.010533,0.000000,0.000000,0.000000,0.016814,0.049696
max,0.408447,0.249029,0.350215,0.363484,0.193550,0.380318,0.249853,0.135410,0.450141,0.286796,...,0.226141,0.358545,0.173914,0.250255,0.261881,0.224120,0.284136,0.242280,0.427309,0.268664


#### Save vectorized DataFrame

In [7]:
#vectorized_str_cols.to_csv(r'D:\vectorized_str_cols-VAR_0493-VAR_0404-VAR_0200.csv')
vectorized_str_cols = pd.read_csv(r'D:\vectorized_str_cols-VAR_0493-VAR_0404-VAR_0200.csv')

#### Delete non-vectorized columns and join vectorized Dataframe with the original DataFrame

In [8]:
df = df.drop(cols_to_vectorize, axis=1)
df = df.join(vectorized_str_cols)

In [21]:
cleaned_str_cols = [c for c in cleaned_str_cols if c not in cols_to_vectorize]
cleaned_str_cols.extend(vectorized_str_cols.columns.tolist())

### One-hot-encode string columns with less than 300 values

In [22]:
columns_to_encode = [c for c in cleaned_str_cols if c not in vectorized_str_cols.columns]
columns_to_encode

[]

In [14]:
dictVectorizer = DictVectorizer(sparse=False)
encoded_str_cols = dictVectorizer.fit_transform(df[columns_to_encode].to_dict(orient='records'))
vec_df = pd.DataFrame(encoded_str_cols, columns=dictVectorizer.get_feature_names(), index=df.index, dtype=np.float64)

KeyError: "[u'VAR_0001' u'VAR_0005' u'VAR_0214' u'VAR_0237' u'VAR_0274' u'VAR_0283'\n u'VAR_0305' u'VAR_0325' u'VAR_0342' u'VAR_0352' u'VAR_0353' u'VAR_0354'\n u'VAR_0466' u'VAR_0467' u'VAR_1934'] not in index"

#### Delete non-encoded columns and join encoded Dataframe with the original DataFrame

In [12]:
df = df.drop(columns_to_encode, axis=1)
df = df.join(vec_df)

In [23]:
cleaned_str_cols = vec_df.columns.tolist() + vectorized_str_cols.columns.tolist()

## Datetime columns
### Create dataframes to separately store the year, month and day information of the date columns

In [15]:
df[cleaned_date_cols] = df[cleaned_date_cols].astype('datetime64[ns]')
years = pd.DataFrame(columns=[c+'_year' for c in cleaned_date_cols], index=df.index, dtype=np.float64)
months = pd.DataFrame(columns=[c+'_month' for c in cleaned_date_cols], index=df.index, dtype=np.float64)
days = pd.DataFrame(columns=[c+'_day' for c in cleaned_date_cols], index=df.index, dtype=np.float64)

for c in cleaned_date_cols:
    dateIndex = pd.DatetimeIndex(df[c])
    years[c+'_year'] = dateIndex.year
    months[c+'_month'] = dateIndex.month
    days[c+'_day'] = dateIndex.day

### Delete the original date columns and join the years, months and days DataFrames with the original DataFrame

In [16]:
df = df.drop(cleaned_date_cols, axis=1)
df = df.join(years)
df = df.join(months)
df = df.join(days)

In [17]:
import itertools
cleaned_date_cols = zip(years.columns, months.columns, days.columns)
cleaned_date_cols = list(itertools.chain(*cleaned_date_cols))
cleaned_date_cols

[]

## Somewhere along the way the index is added as a column twice. Delete it.

In [1]:
df = df.drop(['Unnamed: 0', 'ID'], axis=1)

NameError: name 'df' is not defined

# Eliminate columns with 0 variance

## Convert everything to np.float64

In [30]:
non_float_cols = df.dtypes[df.dtypes != np.float64].index.tolist()
df[non_float_cols] = df[non_float_cols].astype(np.float64)

## See which columns have low standard deviation

In [33]:
df_desc = df.describe()
df_desc[sorted(df_desc.columns, key=lambda x: df_desc.loc['std', x])]

,VAR_0204_year,VAR_0526,VAR_0106,VAR_0191,VAR_0192,VAR_0098,VAR_0138,VAR_0130,VAR_0114,VAR_0193,...,VAR_1892,VAR_1200,VAR_1202,VAR_1199,VAR_1204,VAR_1203,VAR_1220,VAR_0931,VAR_1221,VAR_0934
count,145231,145231.000000,145231.000000,145231.000000,145231.000000,145231.000000,145231.000000,145231.000000,145231.000000,145231.000000,...,1.452310e+05,1.452310e+05,1.452310e+05,1.452310e+05,1.452310e+05,1.452310e+05,1.452310e+05,1.452310e+05,1.452310e+05,1.452310e+05
mean,2014,0.000007,0.000028,0.000028,0.000090,0.000131,0.000131,0.000145,0.000152,0.000152,...,4.488858e+08,4.687557e+08,4.687536e+08,4.687520e+08,4.687432e+08,4.691248e+08,4.798996e+08,4.798984e+08,4.798942e+08,4.798909e+08
std,0,0.002624,0.005248,0.005248,0.010819,0.011437,0.011437,0.012024,0.012307,0.015300,...,4.973813e+08,4.990118e+08,4.990137e+08,4.990152e+08,4.990235e+08,4.990374e+08,4.995886e+08,4.995898e+08,4.995938e+08,4.995970e+08
min,2014,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,5.000000e+02,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,2014,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000e+03,1.911800e+04,1.804850e+04,1.348800e+04,4.000000e+02,1.731300e+04,1.381000e+04,1.301950e+04,3.730000e+03,3.800000e+02
50%,2014,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.750000e+03,5.248600e+04,3.663800e+04,4.035200e+04,1.014000e+03,3.388900e+04,4.587100e+04,3.351700e+04,2.416400e+04,6.980000e+02
75%,2014,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09
max,2014,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,3.000000,...,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09


## Eliminate all columns with standard deviation lower than 0.02 (seems like a reasonable value)

In [35]:
std_series = df_desc.loc['std', :]
std_series[std_series < 0.02]
#df = df.drop(( ).columns, axis=1)

VAR_0098                                     0.011437
VAR_0106                                     0.005248
VAR_0107                                     0.018178
VAR_0114                                     0.012307
VAR_0130                                     0.012024
VAR_0138                                     0.011437
VAR_0191                                     0.005248
VAR_0192                                     0.010819
VAR_0193                                     0.015300
VAR_0194                                     0.019459
VAR_0392                                     0.017403
VAR_0395                                     0.004545
VAR_0396                                     0.008703
VAR_0397                                     0.009461
VAR_0398                                     0.009818
VAR_0399                                     0.013634
VAR_0411                                     0.008703
VAR_0412                                     0.017988
VAR_0459                    

# Save preprocessed data to another csv file

In [32]:
df.to_csv(PTRAIN_PATH, index=False)

In [35]:
df.head()

,ID,VAR_0001,VAR_0002,VAR_0003,VAR_0004,VAR_0005,VAR_0006,VAR_0007,VAR_0013,VAR_0014,...,VAR_0166_day,VAR_0167_day,VAR_0168_day,VAR_0169_day,VAR_0176_day,VAR_0177_day,VAR_0178_day,VAR_0179_day,VAR_0204_day,VAR_0217_day
0,2,h,224,0,4300,c,0,0,0,0,...,18,2,2,18,11,2,2,3,29,8
1,4,h,7,53,4448,b,1,0,1,0,...,18,2,2,18,11,2,2,3,31,2
2,5,h,116,3,3464,c,0,0,0,0,...,18,2,2,18,11,2,2,3,30,13
3,7,h,240,300,3200,c,0,0,0,0,...,18,2,2,18,11,2,2,3,31,1
4,8,r,72,261,2000,n,0,0,0,0,...,18,2,2,18,11,2,2,3,29,15
